In [1]:
# Read the CSV file 'UKB_olink_proteins.csv' into a DataFrame
import pandas as pd
df = pd.read_csv('UKB_olink_proteins.csv')
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# Merge dataframes df and df1 based on the 'eid' column
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
import operator
import warnings
warnings.filterwarnings(('ignore'))


df1= pd.read_csv('pain_onset.csv')

mydf1 = pd.merge(df, df1, how='inner', on=['eid'])

mydf1.head()

In [ ]:
# Replace -9 in column A with the vacancy value NaN
mydf1['back'].replace(-9, np.nan, inplace=True)

mydf2=mydf1

mydf2.head()

# Delete the row with the vacancy value in column A
mydf2.dropna(subset=['back'], inplace=True)

mydf2.shape

In [ ]:
# Remove specified features from the dataframe
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings(('ignore'))

rm_f1 = ['eid', 'ins_index']
rm_HES = ['head', 'face', 'neck', 'back', 'stomach','hip','knee']
rm_f = rm_f1 + rm_HES

X = mydf2.drop(rm_f, axis=1)
y = mydf2['head']

In [ ]:
# Perform stratified K-fold cross-validation with 5 splits, and shuffle the data
mykf = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle = True)

my_params = {'n_estimators': 500,
             'max_depth': 15,
             'num_leaves': 10,
             'subsample': 0.7,
             'learning_rate': 0.01,
             'colsample_bytree': 0.7}

def normal_imp(mydict):
    mysum = sum(mydict.values())
    mykeys = mydict.keys()
    for key in mykeys:
        mydict[key] = mydict[key]/mysum
    return mydict

tg_imp_cv = Counter()
tc_imp_cv = Counter()

for train_idx, test_idx in mykf.split(X, y):
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]
    my_lgb = LGBMClassifier(objective = 'binary',
                           metric = 'auc',
                           is_unbalance = True,
                           verbosity = 1, seed = 2020)
    my_lgb.set_params(**my_params)
    my_lgb.fit(X_train, y_train)
    totalgain_imp = my_lgb.booster_.feature_importance(importance_type='gain')
    totalgain_imp = dict(zip(my_lgb.booster_.feature_name(), totalgain_imp.tolist()))
    totalcover_imp = my_lgb.booster_.feature_importance(importance_type='split')
    totalcover_imp = dict(zip(my_lgb.booster_.feature_name(), totalcover_imp.tolist()))
    tg_imp_cv += Counter(normal_imp(totalgain_imp))
    tc_imp_cv += Counter(normal_imp(totalcover_imp))


tg_imp_df = pd.DataFrame({'Features': list(tg_imp_cv.keys()),
                          'TotalGain_cv': list(tg_imp_cv.values())})

tc_imp_df = pd.DataFrame({'Features': list(tc_imp_cv.keys()),
                          'TotalCover_cv': list(tc_imp_cv.values())})


In [ ]:
my_imp_df = pd.merge(left = tc_imp_df, right = tg_imp_df, how = 'left')
my_imp_df.sort_values(by = 'TotalGain_cv', ascending = False, inplace = True)

my_imp_df.head()

In [ ]:
myout=my_imp_df
myout.to_csv('result_pain/back_s01.csv')

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
import operator
import warnings
warnings.filterwarnings(('ignore'))

df_f = pd.read_csv('result_pain/back_s01.csv')
df_f.sort_values(by = 'TotalCover_cv', ascending = False, inplace = True)
df_f1 = df_f.head(100)
my_f = df_f1['Features'].to_list()

X = mydf2[my_f]
y = mydf2['back']

In [ ]:
# Train a LightGBM classifier using stratified K-fold cross-validation with 5 splits, and calculate AUC metrics
mykf = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle = True)


my_params = {'n_estimators': 875,
             'max_depth': 16,
             'num_leaves': 11,
             'subsample': 0.7,
             'learning_rate': 0.01,
             'colsample_bytree': 0.7}


tmp_f, AUC_cv_lst= [], []

for f in my_f:
    tmp_f.append(f)
    my_X = X[tmp_f]
    AUC_cv = []
    for train_idx, test_idx in mykf.split(my_X, y):
        X_train, X_test = my_X.iloc[train_idx, :], my_X.iloc[test_idx, :]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        my_lgb = LGBMClassifier(objective='binary',
                                metric='auc',
                                is_unbalance=True,
                                n_jobs=4,
                                verbosity=-1, seed=2022)
        my_lgb.set_params(**my_params)
        my_lgb.fit(X_train, y_train)
        y_pred_prob = my_lgb.predict_proba(X_test)[:, 1]
        AUC_cv.append(roc_auc_score(y_test, y_pred_prob))
    tmp_out = np.array([np.mean(AUC_cv), np.std(AUC_cv)] + AUC_cv)
    AUC_cv_lst.append(np.round(tmp_out, 3))
    print((f, tmp_out))

In [ ]:
AUC_df = pd.concat((pd.DataFrame({'Features':tmp_f}), pd.DataFrame(AUC_cv_lst)), axis = 1)

AUC_df.head()

myout2=AUC_df

myout2.columns = ['Features', 'AUC_mean', 'AUC_std', 'AUC0', 'AUC1', 'AUC2', 'AUC3', 'AUC4']

myout2.tail()

In [ ]:
myout2.to_csv('result_pain/back_s02.csv')
df_f2 = df_f.head(50)
df_f2.to_csv('result_pain/back_s001.csv')

df_f3 = pd.read_csv('result_pain/back_s02.csv')
df_f3 = df_f3.head(50)
df_f3.shape
df_f3.to_csv('result_pain/back_s002.csv')

In [ ]:
# Visualize predictor importance (Fimp) and cumulative AUC values for prediction
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns


fimp_s03 = pd.read_csv('result_pain/back_s001.csv')
fimp_s03.sort_values(by = 'TotalCover_cv', ascending = False, inplace = True)
fimp_s04 = pd.read_csv('result_pain/back_s002.csv')
fimp_s04['f_idx'] = fimp_s04['Unnamed: 0'] + 1
mylabel = (fimp_s03['Features'])

fimp_tc = pd.DataFrame(zip(fimp_s03['TotalCover_cv'], mylabel), columns=['Fimp','Feature'])
fimp_tc = pd.concat((fimp_s04[['f_idx', 'AUC_mean', 'AUC0', 'AUC1', 'AUC2', 'AUC3', 'AUC4', 'AUC_std']], fimp_tc), axis = 1)
fimp_tc['AUC_lower'] = fimp_tc['AUC_mean'] - 1.96*fimp_tc['AUC_std']
fimp_tc['AUC_upper'] = fimp_tc['AUC_mean'] + 1.96*fimp_tc['AUC_std']

fig, ax = plt.subplots(figsize = (18, 7))
palette = sns.color_palette("Blues",n_colors=len(fimp_tc))
palette.reverse()
sns.barplot(ax=ax, x = "Feature", y = "Fimp", palette=palette, data=fimp_tc.sort_values(by="Fimp", ascending=False))
ax.set_ylim([0, 0.1])
ax.tick_params(axis='y', labelsize=14)
ax.set_xticklabels(fimp_tc['Feature'], rotation=30, fontsize=12, horizontalalignment='right')
nb_f = 23
my_col = ['r']*nb_f + ['k']*(len(fimp_tc)-nb_f)
for ticklabel, tickcolor in zip(plt.gca().get_xticklabels(), my_col):
    ticklabel.set_color(tickcolor)
ax.set_ylabel('Predictor Importance', weight='bold', fontsize=18)
#ax.set_title('10-year incident AD', y=1.0, pad=-25, weight='bold', fontsize=24)
ax.set_xlabel('')
ax.grid(which='minor', alpha=0.2, linestyle=':')
ax.grid(which='major', alpha=0.5,  linestyle='--')
ax.set_axisbelow(True)

ax2 = ax.twinx()
#ax2.plot(fimp_tc['f_idx']-1, fimp_tc['AUC0'], 'mediumvioletred', alpha = 0.15, marker='o')
#ax2.plot(fimp_tc['f_idx']-1, fimp_tc['AUC1'], 'mediumvioletred', alpha = 0.15, marker='o')
#ax2.plot(fimp_tc['f_idx']-1, fimp_tc['AUC2'], 'mediumvioletred', alpha = 0.15, marker='o')
#ax2.plot(fimp_tc['f_idx']-1, fimp_tc['AUC3'], 'mediumvioletred', alpha = 0.15, marker='o')
#ax2.plot(fimp_tc['f_idx']-1, fimp_tc['AUC4'], 'mediumvioletred', alpha = 0.15, marker='o')
ax2.plot(np.arange(nb_f+1), fimp_tc['AUC_mean'][:nb_f+1], 'red', alpha = 0.8, marker='o')
ax2.plot(np.arange(nb_f+1, len(fimp_tc)), fimp_tc['AUC_mean'][nb_f+1:], 'black', alpha = 0.8, marker='o')
ax2.plot([nb_f, nb_f+1], fimp_tc['AUC_mean'][nb_f:nb_f+2], 'black', alpha = 0.8, marker='o')
plt.fill_between(fimp_tc['f_idx']-1, fimp_tc['AUC_lower'], fimp_tc['AUC_upper'], color = 'tomato', alpha = 0.2)
ax2.set_ylabel('Cumulative AUC', weight='bold', fontsize=18)
ax2.tick_params(axis='y', labelsize=14)
#ax2.set_yticklabels([0.78, 0.80, 0.82, 0.84, 0.86], fontsize=14)
fig.tight_layout()
plt.xlim([-.6, len(fimp_tc)-.2])
plt.savefig('result_pain/Cover_Imp_back.png')

In [ ]:
# Plot the mean ROC curve 
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

plt.figure(figsize = (12, 12))

# my_X is the feature data, y is the label data, and my_params is the model parameter
mykf = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle = True)
my_params = {'n_estimators': 1000,
             'max_depth': 5,
             'num_leaves': 30,
             'subsample': 0.7,
             'learning_rate': 0.01,
             'colsample_bytree': 0.7}

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

# Loop through the feature data
for train_idx, test_idx in mykf.split(X, y):
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    my_lgb = LGBMClassifier(objective='binary', metric='auc', is_unbalance=True, n_jobs=4, verbosity=-1, seed=2022)
    my_lgb.set_params(**my_params)
    my_lgb.fit(X_train, y_train)
    y_pred_prob = my_lgb.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    aucs.append(roc_auc)

# Calculate the average ROC curve and AUC values
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)


tprs_upper = np.minimum(mean_tpr + 2*std_auc, 1)
tprs_lower = mean_tpr - 2*std_auc

plt.plot(mean_fpr,  mean_tpr, linewidth = 5,label=r'Mean ROC (AUC = %0.2f)' % (mean_auc))

plt.legend(loc="lower right",fontsize = 20)


plt.plot([0, 1], [0, 1], 'k--',linewidth=5 )
plt.xlim([-0.0, 1.0])
plt.ylim([-0.0, 1.0])
plt.ylabel('True Positive Rate', fontsize = 30)
plt.xlabel('False Positive Rate', fontsize = 30)

plt.yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], fontsize = 16)
plt.xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], fontsize = 16)
plt.grid(which='minor', alpha=0.2, linestyle=':')
plt.grid(which='major', alpha=0.5,  linestyle='--')
plt.tight_layout()
plt.savefig('result_pain/ROC_back.pdf')